<a href="https://colab.research.google.com/github/ekaterina533/dataset/blob/main/%D0%9F%D1%80%D0%B8%D0%BB%D0%BE%D0%B6%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%BF%D0%B5%D1%80%D0%B5%D0%BC%D0%B5%D1%89%D0%B5%D0%BD%D0%B8%D0%B5%20%D1%82%D0%B5%D1%81%D1%82%D0%B8%D1%80%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ldap3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.2/432.2 kB 1.8 MB/s eta 0:00:00


In [2]:
%%writefile core.py
import os
import logging
from logging.handlers import RotatingFileHandler
import json

class Config(object):

    def __init__(self, kwargs, log_level: str='DEBUG', path_to_export_file: str='./', ldap_server: str=None, ldap_user: str=None, ldap_password: str=None, ldap_bind: str=None, email_login: str='test@avt-stv.ru', email_password: str='ddd', email_server: str='avt-stv.ru', mail_send_to: str='it@avt-stv.ru', ignore_on_departament_change_dn_group: list=[]):
        self.dry_run = kwargs.get('dry_run',False)
        self.log_level = log_level
        self.path_to_export_file = path_to_export_file
        self.ldap_server = ldap_server
        self.ldap_user = ldap_user
        self.ldap_password = ldap_password
        self.ldap_bind = ldap_bind
        self.email_login = email_login
        self.email_password = email_password
        self.email_server = email_server
        self.mail_send_to = mail_send_to
        self.ignore_on_departament_change_dn_group = ignore_on_departament_change_dn_group

    def read_conf(cls):
        if not os.path.exists('config.json'):
            data = json.dumps(cls.__dict__)
            with open('config.json', 'w') as f:
                f.write(data)
        with open('config.json', encoding='utf-8') as f:
            data = json.load(f)
            for item in dir(cls):
                if not callable(getattr(cls, item)) and (not item.startswith('__')):
                    try:
                        setattr(cls, item, data[item])
                    except KeyError:
                        pass
            return cls
config = Config().read_conf()
logging.basicConfig(handlers=[RotatingFileHandler('log', mode='a', maxBytes=5242880, backupCount=0, encoding='utf-8', delay=0), logging.StreamHandler()], level=logging.getLevelName(config.log_level), format='%(asctime)s.%(msecs)03d %(levelname)s:%(message)s', datefmt='%d.%m.%Y %H:%M:%S')

Writing core.py


In [3]:
%%writefile onec.py

from core import config, logging
from objects import File_user_object
import json

def import_users_from_file(ad_ous):
    logging.info('Ищем пользователей в 1C')
    users = []
    f = open(config.path_to_export_file, encoding='utf-8')
    data = json.load(f)
    for item in data:
        name = item['ФИО'].split(' ')
        try:
            first_name = name[1]
        except IndexError:
            first_name = None
        try:
            last_name = name[0]
        except IndexError:
            last_name = None
        try:
            middle_name = name[2]
        except IndexError:
            middle_name = None
        director_name = item['Руководитель'].split(' ')
        try:
            director_first_name = director_name[1]
        except IndexError:
            director_first_name = None
        try:
            director_last_name = director_name[0]
        except IndexError:
            director_last_name = None
        try:
            director_middle_name = director_name[2]
        except IndexError:
            director_middle_name = None
        code = int(item['Код'])
        division = item['ПодразделениеУчасток']
        departament = item['Подразделение']
        post = item['Должность']
        number = item['Телефон']
        status = item['Статус']
        if status == 'работает':
            status = True
        else:
            status = False
        departament_code = int(item['КодПодразделения'])
        division_code = int(item['КодПодразделенияУчастка'])
        director_code = int(item['КодРуководителя'])
        division_OU = None
        for ad_ou in ad_ous:
            if ad_ou.code == division_code:
                division_OU = ad_ou
                break
        departament_OU = None
        for ad_ou in ad_ous:
            if ad_ou.code == departament_code:
                departament_OU = ad_ou
                break
        users.append(File_user_object(departament_OU=departament_OU, division_OU=division_OU, departament_code=departament_code, division_code=division_code, director_code=director_code, first_name=first_name, last_name=last_name, middle_name=middle_name, director_first_name=director_first_name, director_last_name=director_last_name, director_middle_name=director_middle_name, code=code, division=division, departament=departament, post=post, number=number, works=status))
    logging.info(f'Найдено {len(users)} пользователей')
    return users

Writing onec.py


In [4]:
%%writefile gost_translate.py

def gost_translate(first_name, middle_name, last_name):
    TRANSLITERATE_DICT = {'а':'a','б':'b','в':'v','г':'g','д':'d','е':'e','ё':'yo',
      'ж':'zh','з':'z','и':'i','й':'i','к':'k','л':'l','м':'m','н':'n',
      'о':'o','п':'p','р':'r','с':'s','т':'t','у':'u','ф':'f','х':'h',
      'ц':'c','ч':'ch','ш':'sh','щ':'sch','ъ':'','ы':'y','ь':'','э':'e',
      'ю':'u','я':'ya', 'А':'A','Б':'B','В':'V','Г':'G','Д':'D','Е':'E','Ё':'YO',
      'Ж':'ZH','З':'Z','И':'I','Й':'I','К':'K','Л':'L','М':'M','Н':'N',
      'О':'O','П':'P','Р':'R','С':'S','Т':'T','У':'U','Ф':'F','Х':'H',
      'Ц':'C','Ч':'CH','Ш':'SH','Щ':'SCH','Ъ':'','Ы':'y','Ь':'','Э':'E',
      'Ю':'U','Я':'YA',',':'','?':'',' ':'_','~':'','!':'','@':'','#':'',
      '$':'','%':'','^':'','&':'','*':'','(':'',')':'','-':'','=':'','+':'',
      ':':'',';':'','<':'','>':'','\'':'','"':'','\\':'','/':'','№':'',
      '[':'',']':'','{':'','}':'','ґ':'','ї':'', 'є':'','Ґ':'g','Ї':'i',
      'Є':'e', '—':''}

    result = ''
    input = f'{last_name}.{first_name}'
    for char in input:
        char_copy = char.upper()
        if char_copy in TRANSLITERATE_DICT:
            if char.isupper():
                char = TRANSLITERATE_DICT[char_copy].capitalize()
            else:  # inserted
                char = TRANSLITERATE_DICT[char_copy].lower()
        result += char
    return result

Writing gost_translate.py


In [5]:
%%writefile mail.py
import smtplib
from email.message import EmailMessage
from core import config

def message_content_generate(content):
    msg = EmailMessage()
    msg['From'] = config.email_login
    msg['To'] = config.mail_send_to
    msg_content = f'\n    <!DOCTYPE html>\n    <html>\n        <body>\n            <div>\n                {content}\n            </div>\n        </body>\n    </html>\n    '
    msg.set_content(msg_content, subtype='html')
    return msg

def send_to_mail(msg: EmailMessage):
    mail = smtplib.SMTP(config.email_server, 25)
    mail.ehlo()
    mail.send_message(msg=msg)
    mail.close()

def empty_code_alert(empty_user_code):
    if len(empty_user_code) == 0:
        return
    text = []
    for user in empty_user_code:
        text.append(f'<h5>Логин: {user.login}</h5>')
    text = '\n'.join(text)
    text = f'<h4>Следующие учетные записи не имеют присвоенного кода физ. лица</h4>{text}<h6>Если код физ. лица не подразумевается для учетной записиси, впишите 000000</h6>'
    msg = message_content_generate(text)
    msg['Subject'] = 'Учетные записи без кода физ.лица'
    send_to_mail(msg=msg)

def empty_code_alert_group(empty_group_code):
    if len(empty_group_code) == 0:
        return
    text = []
    for group in empty_group_code:
        text.append(f'<h5>DN: {group.dn}</h5>')
    text = '\n'.join(text)
    text = f'<h4>Следующие группы не имеют присвоенного кода </h4>{text}<h6>Если код не подразумевается для группы, впишите 000000</h6>'
    msg = message_content_generate(text)
    msg['Subject'] = 'Группы записи без кода'
    send_to_mail(msg=msg)

def empty_code_alert_ou(empty_ou_code):
    if len(empty_ou_code) == 0:
        return
    text = []
    for ou in empty_ou_code:
        text.append(f'<h5>DN: {ou.dn}</h5>')
    text = '\n'.join(text)
    text = f'<h4>Следующие OU не имеют присвоенного кода </h4>{text}<h6>Если код не подразумевается для OU, впишите 000000</h6>'
    msg = message_content_generate(text)
    msg['Subject'] = 'OU записи без кода'
    send_to_mail(msg=msg)

def fio_change_alert(user, new_login, old_fio, new_fio, error_code):
    if error_code == None:
        text = f'<h4>Произошла смена ФИО учетной записи {user.login}</h4><h4>{old_fio} > {new_fio}</h4><h5>Предпологаемый новый логин: {new_login}</h5><h6>Логин не был изменен, смена производится вручную с уведомлением пользователя</h6>'
    else:
        text = f'<h4>Не произошла смена ФИО учетной записи {user.login}</h4><h4>{old_fio} > {new_fio}</h4><h5>Предпологаемый новый логин: {new_login}</h5><h6>{error_code}</h6>'
    msg = message_content_generate(text)
    if error_code == None:
        msg['Subject'] = f'Смена ФИО {user.login}'
    else:
        msg['Subject'] = f'Смена ФИО {user.login} [ОШИБКА]'
    send_to_mail(msg=msg)

def post_change_alert(user, new_post):
    text = f'<h4>Произошла смена должности учетной записи {user.login}</h4><h4>{user.post} > {new_post}</h4>'
    msg = message_content_generate(text)
    msg['Subject'] = f'Смена должности {user.login}'
    send_to_mail(msg=msg)

def divison_not_found_alert(empty_divisions):
    if len(empty_divisions) == 0:
        return
    text = []
    for item in empty_divisions:
        text_tmp = f'<h5>Название: {item.division}, Код: {item.division_code}</h5>'
        if text_tmp not in text:
            text.append(text_tmp)
    text = '\n'.join(text)
    text = f'<h4>Не найдены подразделения в AD:</h4>{text}'
    msg = message_content_generate(text)
    msg['Subject'] = 'Отутствуют подразделения'
    send_to_mail(msg=msg)

def ou_not_found_alert(user, ou):
    text = f'<h4>Не найден OU для учетной записи {user.login}</h4><h4>{ou}</h4>'
    msg = message_content_generate(text)
    msg['Subject'] = f'Смена OU {user.login}'
    send_to_mail(msg=msg)

def division_change_alert(ad_user, file_user):
    old_dn = ','.join(ad_user.dn.split(',')[1:])
    text = f'<h4>Подразделение для учетной записи {ad_user.login} изменено на {file_user.division}</h4><h4>Удалена группа: {ad_user.division_OU.main_group}</h4><h4>Добавлена группа: {file_user.division_OU.main_group}</h4><h4>Перемещен из: {old_dn}</h4><h4>Перемещен в: {file_user.division_OU.dn}</h4>'
    msg = message_content_generate(text)
    msg['Subject'] = f'Смена подразделения {ad_user.login}'
    send_to_mail(msg=msg)

def departament_change_alert(ad_user, file_user, removed_groups):
    text = []
    for remove_group in removed_groups:
        text.append(f'<h4>{remove_group}</h4>')
    text = '\n'.join(text)
    text = f'<h4>Отдел для учетной записи {ad_user.login} изменен на {file_user.division}</h4><h4>Удалены группы:</h4>{text}<h4>Старый отдел: {ad_user.departament_OU.dn}</h4><h4>Новый отдел: {file_user.departament_OU.dn}</h4>'
    msg = message_content_generate(text)
    msg['Subject'] = f'Смена отдела {ad_user.login}'
    send_to_mail(msg=msg)

def disable_users_alert(ad_user):
    text = f'<h4>Учетная запись {ad_user.login} отключена в связи с увольнением</h4>'
    msg = message_content_generate(text)
    msg['Subject'] = f'Учетная запись отключена {ad_user.login}'
    send_to_mail(msg=msg)

def send_error(error):
    text = f'<h4>{error}</h4>'
    msg = message_content_generate(text)
    msg['Subject'] = 'ОШИБКА!!!'
    send_to_mail(msg=msg)

Writing mail.py


In [6]:
%%writefile main.py

from core import logging, config
from mail import empty_code_alert, empty_code_alert_group, empty_code_alert_ou, divison_not_found_alert, disable_users_alert, send_error
from onec import import_users_from_file
from ad import import_users_from_domain, update_FIO, update_company, update_post, update_number, get_all_ou, get_all_groups, departament_change, division_change, disable_user
import traceback
from ldap3 import SAFE_SYNC, Connection, Server, SUBTREE, ALL_ATTRIBUTES, ServerPool, MODIFY_REPLACE

def update_code(user):
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')
    with Connection(server_pool, config.ldap_user, config.ldap_password, client_strategy=SAFE_SYNC, auto_bind=True) as c:
        c.modify(user.dn, {'info': [(MODIFY_REPLACE, [0])]})
        logging.debug(c.result)
    return True

def main():
    ad_groups = get_all_groups()
    empty_group_code = []
    for ad_group in ad_groups:
        if ad_group.code == None:
            empty_group_code.append(ad_group)
            logging.info(f'Найдена группа без кода {ad_group.dn}')
    empty_code_alert_group(empty_group_code=empty_group_code)
    ad_ous = get_all_ou(ad_gropus=ad_groups)
    empty_ou_code = []
    for ad_ou in ad_ous:
        if ad_ou.code == None:
            empty_ou_code.append(ad_ou)
            logging.info(f'Найдена OU без кода {ad_ou.dn}')
    empty_code_alert_ou(empty_ou_code=empty_ou_code)
    file_users = import_users_from_file(ad_ous=ad_ous)
    ad_users = import_users_from_domain(ad_ous=ad_ous)
    empty_user_code = []
    for ad_user in ad_users:
        if ad_user.code == None and ad_user.is_lock == False:
            empty_user_code.append(ad_user)
            logging.info(f'Найден AD юзер без кода {ad_user.dn}')
    empty_code_alert(empty_user_code=empty_user_code)
    empty_divisions = []
    for file_user in file_users:
        if file_user.division_OU == None:
            empty_divisions.append(file_user)
            logging.info(f'Найден 1С юзер без подразделения {file_user.FIO}')
    divison_not_found_alert(empty_divisions=empty_divisions)
    for file_user in file_users:
        for ad_user in ad_users:
            if file_user.director_code == ad_user.code:
                file_user.director_obj = ad_user
                break
    for ad_user in ad_users:
        for director_ad_user in ad_users:
            if ad_user.director_dn == director_ad_user.dn:
                ad_user.director_obj = director_ad_user
                break
    for ad_user in ad_users:
        if ad_user.code == 0:
            continue
        found_flag = False
        for file_user in file_users:
            if file_user.code == ad_user.code:
                found_flag = True
                break
        if found_flag == False:
            logging.warning(f'User not found in ad{ad_user.dn}')
            continue
        if ad_user.first_name != file_user.first_name or ad_user.last_name != file_user.last_name or ad_user.middle_name != file_user.middle_name:
            ad_user = update_FIO(ad_user=ad_user, file_user=file_user)
        if ad_user.company != 'АО "Электроавтоматика"':
            update_company(ad_user=ad_user)
        if ad_user.post != file_user.post:
            ad_user = update_post(ad_user=ad_user, new_post=file_user.post)
        if ad_user.number != file_user.number and ad_user.number != None and (file_user.number != ''):
            ad_user = update_number(ad_user=ad_user, new_number=file_user.number)
        if ad_user.departament_OU != file_user.departament_OU:
            ad_user = departament_change(ad_user=ad_user, file_user=file_user)
        if ad_user.division_OU != file_user.division_OU or ad_user.division == None or ad_user.director_obj == None or (ad_user.director_obj != file_user.director_obj):
            ad_user = division_change(ad_user=ad_user, file_user=file_user)
        if ad_user.is_lock == False and file_user.works == False:
            disable_user(user=ad_user)
            disable_users_alert(ad_user=ad_user)
if __name__ == '__main__':
    try:
        main()
    except Exception as e:
        send_error(error=traceback.format_exc())
        logging.critical(e, exc_info=True)
    logging.info('Успешный экспорт')

Writing main.py


In [8]:
%%writefile objects.py

from core import logging

class AD_group_object(object):

    def __init__(self, dn: str, code: int) -> None:
        self.dn = dn
        self.code = code
        logging.info(f'Новая группа: {self.dn}, {self.code}')

    def __str__(self):
        return self.dn

class AD_OU_object(object):

    def __init__(self, dn: str, code: int, main_group: AD_group_object) -> None:
        self.dn = dn
        self.code = code
        self.main_group = main_group
        logging.info(f'Новый OU: {self.dn}, {self.code}')

    def __str__(self):
        return self.dn

class AD_user_object(object):

    def __init__(self, description: str, UserAccountControl: int, director_dn: str, division_OU: AD_OU_object, departament_OU: AD_OU_object, first_name: str, last_name: str, middle_name: str, code: int, director_first_name: str, director_last_name: str, director_middle_name: str, division: str, departament: str, post: str, number: str, is_lock: bool, groups: list, company: str, login: str, dn: str, director_obj=None):
        self.first_name = first_name
        self.last_name = last_name
        self.middle_name = middle_name
        self.code = code
        self.director_first_name = director_first_name
        self.director_last_name = director_last_name
        self.director_middle_name = director_middle_name
        self.division = division
        self.departament = departament
        self.post = post
        self.number = number
        self.is_lock = is_lock
        self.groups = groups
        self.company = company
        self.login = login
        self.dn = dn
        self.FIO = f'{last_name} {first_name} {middle_name}'
        self.division_OU = division_OU
        self.departament_OU = departament_OU
        self.director_obj = director_obj
        self.director_dn = director_dn
        self.UserAccountControl = UserAccountControl
        self.description = self.get_user_description(description)
        logging.info(f'Новый пользователь в AD: {self.dn}, {self.code}')

    def get_user_description(self, description):
        if description == None:
            return ''
        return ' '.join(description)

class File_user_object(object):

    def __init__(self, division_OU: AD_OU_object, departament_OU: AD_OU_object, departament_code: int, division_code: int, director_code: int, first_name: str, last_name: str, middle_name: str, code: int, director_first_name: str, director_last_name: str, director_middle_name: str, division: str, departament: str, post: str, number: str, works: bool, director_obj: AD_user_object=None):
        self.first_name = first_name
        self.last_name = last_name
        self.middle_name = middle_name
        self.code = code
        self.director_first_name = director_first_name
        self.director_last_name = director_last_name
        self.director_middle_name = director_middle_name
        self.division = division
        self.departament = departament
        self.post = post
        self.number = number
        self.works = works
        self.departament_code = departament_code
        self.division_code = division_code
        self.director_code = director_code
        self.FIO = f'{last_name} {first_name} {middle_name}'
        self.division_OU = division_OU
        self.departament_OU = departament_OU
        self.director_obj = director_obj
        logging.info(f'Новый пользователь в 1С: {self.FIO}, {self.code}')

Writing objects.py


In [12]:
%%writefile ad.py
from core import config, logging
from objects import AD_user_object, AD_group_object, AD_OU_object, File_user_object
from gost_translate import gost_translate
from mail import fio_change_alert, post_change_alert, division_change_alert, departament_change_alert
from ldap3 import SAFE_SYNC, Connection, Server, SUBTREE, ALL_ATTRIBUTES, ServerPool, MODIFY_REPLACE
from ldap3.extend.microsoft.addMembersToGroups import ad_add_members_to_groups
from ldap3.extend.microsoft.removeMembersFromGroups import ad_remove_members_from_groups
from typing import List, Optional, Dict, Any

def dry_run_wrapper(func):
    """Декоратор для dry-run режима"""
    def wrapper(*args, **kwargs):
        if config.dry_run:
            func_name = func.__name__
            user = args[0] if args else None
            if user and hasattr(user, 'login'):
                logging.info(f"DRY RUN: {func_name} для {user.login}")
            else:
                logging.info(f"DRY RUN: {func_name}")

            if func_name == 'update_FIO':
                old_name = f"{user.last_name} {user.first_name} {user.middle_name}"
                new_name = f"{kwargs.get('file_user').last_name} {kwargs.get('file_user').first_name} {kwargs.get('file_user').middle_name}"
                logging.info(f"Было бы изменено ФИО: {old_name} -> {new_name}")

            elif func_name == 'division_change':
                old_ou = user.division_OU.dn if user.division_OU else None
                new_ou = kwargs.get('file_user').division_OU.dn if kwargs.get('file_user').division_OU else None
                logging.info(f"Было бы перемещение из: {old_ou} в: {new_ou}")

            return args[0] if args else None
        return func(*args, **kwargs)
    return wrapper

def import_users_from_domain(ad_ous: List[AD_OU_object]) -> List[AD_user_object]:
    logging.info('Поиск пользователей в домене')
    if config.dry_run:
        logging.info("DRY RUN: Чтение пользователей из AD (режим только для чтения)")
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')

    users = []
    try:
        with Connection(server_pool, config.ldap_user, config.ldap_password,
                      client_strategy=SAFE_SYNC, auto_bind=True) as conn:

            results = conn.extend.standard.paged_search(
                search_base=config.ldap_bind,
                search_filter='(&(objectclass=user)(!(objectclass=computer)))',
                search_scope=SUBTREE,
                attributes=ALL_ATTRIBUTES,
                paged_size=500
            )

            for entry in results:
                try:
                    if 'dn' not in entry:
                        continue

                    attrs = entry.get('attributes', {})
                    user_dn = entry['dn']

                    if config.dry_run:
                        logging.debug(f"DRY RUN: Найден пользователь: {user_dn}")

                    full_name = attrs.get('cn', '').split()
                    name_parts = {
                        'first_name': full_name[0] if len(full_name) > 0 else None,
                        'middle_name': full_name[1] if len(full_name) > 1 else None,
                        'last_name': full_name[2] if len(full_name) > 2 else None
                    }

                    manager_data = _parse_manager_info(attrs.get('manager'))

                    try:
                        code = int(attrs.get('postalCode', 0))
                    except (ValueError, TypeError):
                        code = None

                    # Определение статуса аккаунта
                    user_control = attrs.get('userAccountControl')
                    is_locked = user_control not in [512, 66048] if user_control is not None else False

                    # Поиск организационных подразделений
                    division_ou, departament_ou = _find_organizational_units(
                        dn=entry['dn'],
                        ad_ous=ad_ous
                    )

                    users.append(AD_user_object(
                        UserAccountControl=user_control,
                        description=attrs.get('description'),
                        director_dn=manager_data['dn'],
                        division_OU=division_ou,
                        departament_OU=departament_ou,
                        dn=entry['dn'],
                        groups=attrs.get('memberOf', []),
                        **name_parts,
                        code=code,
                        director_first_name=manager_data['first_name'],
                        director_last_name=manager_data['last_name'],
                        director_middle_name=manager_data['middle_name'],
                        division=attrs.get('department'),
                        departament=_get_departament_from_dn(attrs.get('distinguishedName')),
                        post=attrs.get('title'),
                        number=attrs.get('telephoneNumber'),
                        is_lock=is_locked,
                        company=attrs.get('company'),
                        login=attrs.get('sAMAccountName')
                    ))
                except Exception as e:
                    logging.error(f'Ошибка обработки пользователя {entry.get("dn")}: {str(e)}')
                    continue

    except Exception as e:
        logging.critical(f'Ошибка подключения к AD: {str(e)}')
        raise

    logging.info(f'Найдено {len(users)} пользователей')
    return users

def _parse_manager_info(manager_dn: Optional[str]) -> Dict[str, Optional[str]]:
    """Разбор имени на компоненты"""
    if not manager_dn:
        return {
            'dn': None,
            'first_name': None,
            'last_name': None,
            'middle_name': None
        }

    try:
        name_parts = manager_dn.split(',')[0].replace('CN=', '').split()
        return {
            'dn': manager_dn,
            'first_name': name_parts[0] if len(name_parts) > 0 else None,
            'last_name': name_parts[2] if len(name_parts) > 2 else None,
            'middle_name': name_parts[1] if len(name_parts) > 1 else None
        }
    except Exception:
        return {
            'dn': manager_dn,
            'first_name': None,
            'last_name': None,
            'middle_name': None
        }

def _find_organizational_units(dn: str, ad_ous: List[AD_OU_object]) -> tuple:
    """Поиск подразделений и отделов для DN пользователя"""
    division_ou = None
    departament_ou = None

    dn_parts = dn.split(',')[1:]  # Skip CN part
    division_path = ','.join(dn_parts)

    # Find division OU (direct parent)
    division_ou = next((ou for ou in ad_ous if ou.dn == division_path), None)

    # Find departament OU (first OU in path)
    for part in dn_parts:
        if part.startswith('OU='):
            departament_name = part[3:]
            departament_ou = next((ou for ou in ad_ous if ou.dn.endswith(departament_name)), None)
            if departament_ou:
                break

    return division_ou, departament_ou

def _get_departament_from_dn(dn: Optional[str]) -> Optional[str]:
    """Извлечение названия отдела"""
    if not dn:
        return None

    for part in dn.split(','):
        if part.startswith('OU='):
            return part[3:]
    return None

@dry_run_wrapper
def update_FIO(ad_user: AD_user_object, file_user: AD_user_object) -> AD_user_object:

    if config.dry_run:
        # В dry-run режиме просто обновляем локальный объект
        ad_user.division_OU = file_user.division_OU
        ad_user.division = file_user.division
        ad_user.director_obj = file_user.director_obj
        ad_user.director_dn = file_user.director_obj.dn if file_user.director_obj else None
        return ad_user

    if not all([file_user.first_name, file_user.last_name]):
        raise ValueError("Необходимо указать имя и фамилию")

    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')

    old_fio = ad_user.FIO
    new_cn = f'{file_user.first_name} {file_user.middle_name or ""} {file_user.last_name}'.strip()
    new_dn = f'CN={new_cn},{",".join(ad_user.dn.split(",")[1:])}'

    try:
        with Connection(server_pool, config.ldap_user, config.ldap_password,
                      client_strategy=SAFE_SYNC, auto_bind=True) as conn:

            # Update attributes
            modify_result = conn.modify(ad_user.dn, {
                'displayName': [(MODIFY_REPLACE, [new_cn])],
                'givenName': [(MODIFY_REPLACE, [file_user.first_name])],
                'sn': [(MODIFY_REPLACE, [file_user.last_name])],
                'initials': [(MODIFY_REPLACE, [file_user.middle_name or ""])]
            })

            if not modify_result:
                raise Exception(f"Не удалось обновить атрибут: {conn.result}")

            # Rename DN if name changed
            if new_cn != ad_user.dn.split(',')[0][3:]:
                conn.modify_dn(ad_user.dn, new_cn)
                if conn.result['result'] != 0:
                    raise Exception(f"не удалось переименовать DN: {conn.result}")

            # Update local object
            ad_user.dn = new_dn
            ad_user.first_name = file_user.first_name
            ad_user.middle_name = file_user.middle_name
            ad_user.last_name = file_user.last_name
            ad_user.FIO = f"{file_user.last_name} {file_user.first_name} {file_user.middle_name}"

            # Generate new login and send notification
            new_login = gost_translate(
                first_name=ad_user.first_name,
                middle_name=ad_user.middle_name,
                last_name=ad_user.last_name
            )

            fio_change_alert(
                user=ad_user,
                new_login=new_login,
                old_fio=old_fio,
                new_fio=ad_user.FIO,
                error_code=None
            )

    except Exception as e:
        logging.error(f"Не удалось изменить имя для {ad_user.dn}: {str(e)}")
        fio_change_alert(
            user=ad_user,
            new_login=None,
            old_fio=old_fio,
            new_fio=f'{file_user.last_name} {file_user.first_name} {file_user.middle_name or ""}'.strip(),
            error_code=str(e)
        )
        raise

    return ad_user
    pass

@dry_run_wrapper
def update_company(ad_user):
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')
    with Connection(server_pool, config.ldap_user, config.ldap_password, client_strategy=SAFE_SYNC, auto_bind=True) as c:
        c.modify(ad_user.dn, {'company': [(MODIFY_REPLACE, ['АО \"Электроавтоматика\"'])]})
        logging.debug(c.result)
    pass

@dry_run_wrapper
def update_post(ad_user, new_post):
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')
    with Connection(server_pool, config.ldap_user, config.ldap_password, client_strategy=SAFE_SYNC, auto_bind=True) as c:
        c.modify(ad_user.dn, {'title': [(MODIFY_REPLACE, [new_post])]})
        logging.debug(c.result)
    post_change_alert(user=ad_user, new_post=new_post)
    ad_user.post = new_post
    return ad_user
    pass

@dry_run_wrapper
def update_number(ad_user, new_number):
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')
    with Connection(server_pool, config.ldap_user, config.ldap_password, client_strategy=SAFE_SYNC, auto_bind=True) as c:
        if new_number == '':
            c.modify(ad_user.dn, {'telephoneNumber': [(MODIFY_REPLACE, [])]})
        else:  # inserted
            c.modify(ad_user.dn, {'telephoneNumber': [(MODIFY_REPLACE, [new_number])]})
        logging.debug(c.result)
    ad_user.number = new_number
    return ad_user
    pass

def get_all_groups() -> List[AD_group_object]:
    """Получение групп Active Directory."""
    logging.info('Получение всех групп AD')
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')
    groups = []
    try:
        with Connection(server_pool, config.ldap_user, config.ldap_password,
                      client_strategy=SAFE_SYNC, auto_bind=True) as conn:

            results = conn.extend.standard.paged_search(
                search_base=config.ldap_bind,
                search_filter='(objectCategory=Group)',
                search_scope=SUBTREE,
                attributes=['dn', 'info'],
                paged_size=500
            )

            for entry in results:
                try:
                    if 'dn' not in entry:
                        continue

                    # Parse group code
                    try:
                        code = int(entry['attributes'].get('info', 0))
                    except (ValueError, TypeError):
                        code = None

                    groups.append(AD_group_object(
                        dn=entry['dn'],
                        code=code
                    ))
                except Exception as e:
                    logging.warning(f'Ошибка обработки групп {entry.get("dn")}: {str(e)}')
                    continue

    except Exception as e:
        logging.critical(f'Ошибка подключения к AD: {str(e)}')
        raise

    logging.info(f'Поиск {len(groups)} групп')
    return groups

def get_all_ou(ad_gropus):
    logging.info('Ищем OU в домене')
    server = Server(config.ldap_server)
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')
    with Connection(server_pool, config.ldap_user, config.ldap_password, client_strategy=SAFE_SYNC, auto_bind=True) as c:
        results = c.extend.standard.paged_search(search_base=config.ldap_bind, search_filter='(objectCategory=organizationalUnit)', search_scope=SUBTREE, attributes=ALL_ATTRIBUTES)
    items = []
    for item in results:
        if 'dn' in item:
            try:
                code = int(item['attributes'].get('postalCode'))
            except:
                code = None
            dn = item['dn']
            ad_group_tmp = None
            for ad_group in ad_gropus:
                if ad_group.code == code and code!= 0:
                    ad_group_tmp = ad_group
                    break
            items.append(AD_OU_object(dn=dn, code=code, main_group=ad_group_tmp))
    logging.info(f'Найдено {len(items)} OU')
    return items

def normalize_dn(dn):
    """Нормализует DN для сравнения"""
    if not dn:
        return ""
    return dn.lower().replace(" ", "").strip()

@dry_run_wrapper
def division_change(ad_user, file_user):
    if config.dry_run:
        # В dry-run режиме просто обновляем локальный объект
        ad_user.division_OU = file_user.division_OU
        ad_user.division = file_user.division
        ad_user.director_obj = file_user.director_obj
        ad_user.director_dn = file_user.director_obj.dn if file_user.director_obj else None
        return ad_user

    # Проверяем, действительно ли нужно перемещение
    if ad_user.division_OU is None or file_user.division_OU is None:
        return ad_user

    # Сравниваем DN с нормализацией
    current_dn = ad_user.division_OU.dn.lower().strip()
    new_dn = file_user.division_OU.dn.lower().strip()

    if current_dn == new_dn:
        return ad_user  # Нет необходимости перемещения

    ad_user_cn = ad_user.dn.split(',')[0]
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')

    with Connection(server_pool, config.ldap_user, config.ldap_password, client_strategy=SAFE_SYNC, auto_bind=True) as c:
        # Обновляем атрибуты
        changes = {
            'department': [(MODIFY_REPLACE, [file_user.division])],
            'manager': [(MODIFY_REPLACE, [file_user.director_obj.dn if file_user.director_obj else ''])]
        }
        c.modify(ad_user.dn, changes)

        # Работа с группами
        if ad_user.division_OU.main_group and ad_user.division_OU.main_group.dn:
            ad_remove_members_from_groups(
                connection=c,
                members_dn=ad_user.dn,
                groups_dn=ad_user.division_OU.main_group.dn,
                fix=True
            )

        if file_user.division_OU.main_group and file_user.division_OU.main_group.dn:
            ad_add_members_to_groups(
                connection=c,
                members_dn=ad_user.dn,
                groups_dn=file_user.division_OU.main_group.dn
            )

        # Реальное перемещение только если OU изменились
        c.modify_dn(ad_user.dn, ad_user_cn, new_superior=file_user.division_OU.dn)
        logging.debug(c.result)

    # Обновляем данные пользователя
    division_change_alert(ad_user=ad_user, file_user=file_user)
    ad_user.dn = f'{ad_user_cn},{file_user.division_OU.dn}'
    ad_user.division = file_user.division
    ad_user.director_obj = file_user.director_obj
    ad_user.director_dn = file_user.director_obj.dn if file_user.director_obj else None
    ad_user.division_OU = file_user.division_OU

    return ad_user
    pass

@dry_run_wrapper
def get_user_groups(ad_user):
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')
    with Connection(server_pool, config.ldap_user, config.ldap_password, client_strategy=SAFE_SYNC, auto_bind=True) as c:
        results = c.extend.standard.paged_search(search_base=ad_user.dn, search_filter='(&(objectclass=user)(!(objectclass=computer)))', search_scope=SUBTREE, attributes=ALL_ATTRIBUTES)
        for item in results:
            user_groups = item['attributes'].get('memberOf')
            return user_groups
    pass


@dry_run_wrapper
def remove_ad_groups(ad_user, user_groups):
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')
    with Connection(server_pool, config.ldap_user, config.ldap_password, client_strategy=SAFE_SYNC, auto_bind=True) as c:
        tt = ad_remove_members_from_groups(connection=c, members_dn=ad_user.dn, groups_dn=user_groups, fix=True)
    pass

@dry_run_wrapper
def departament_change(ad_user, file_user):
    if ad_user.division_OU!= file_user.division_OU:
        ad_user = division_change(ad_user=ad_user, file_user=file_user)
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')
    with Connection(server_pool, config.ldap_user, config.ldap_password, client_strategy=SAFE_SYNC, auto_bind=True) as c:
        c.modify(ad_user.dn, {'department': [(MODIFY_REPLACE, [file_user.division])], 'manager': [(MODIFY_REPLACE, [file_user.director_obj.dn])]})
        logging.debug(c.result)
    removed_groups = []
    for user_group in ad_user.groups:
        if user_group in config.ignore_on_departament_change_dn_group or user_group == file_user.division_OU.main_group.dn:
            continue
        removed_groups.append(user_group)
    for remove_group in removed_groups:
        ad_user.groups.remove(remove_group)
    remove_ad_groups(ad_user=ad_user, user_groups=removed_groups)
    departament_change_alert(ad_user=ad_user, file_user=file_user, removed_groups=removed_groups)
    return ad_user
    pass

@dry_run_wrapper
def disable_user(user: AD_user_object) -> bool:
    logging.info(f'Отключаем пользователя {user.login} {user.dn}')
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')

    new_description = f'{user.description or ""} Уволен'.strip()
    lock = 514 if user.UserAccountControl in [512, 66048] else 66050

    with Connection(server_pool, config.ldap_user, config.ldap_password,
                  client_strategy=SAFE_SYNC, auto_bind=True) as c:

        c.modify(user.dn, {
            'UserAccountControl': [(MODIFY_REPLACE, [lock])],
            'description': [(MODIFY_REPLACE, [new_description])]
        })
        logging.debug(c.result)

    return c.result['result'] == 0
    pass

def check_real_system():
    """Функция для проверки на реальной системе"""
    logging.info("=== ТЕСТИРОВАНИЕ НА РЕАЛЬНОЙ СИСТЕМЕ ===")
    logging.info("Режим dry-run: Только чтение и логирование")

    # 1. Чтение данных
    logging.info("1. Чтение групп и подразделений...")
    groups = get_all_groups()
    ous = get_all_ou(groups)
    users = import_users_from_domain(ous)

    if not users:
        logging.error("Не найдено пользователей для тестирования")
        return

    # 2. Выбор тестового пользователя
    test_user = next((u for u in users if u.login == "test_user"), users[0])
    logging.info(f"2. Выбран тестовый пользователь: {test_user.FIO} ({test_user.login})")

    # 3. Подготовка тестовых данных
    test_data = File_user_object(
        first_name=test_user.first_name + "_TEST",
        last_name=test_user.last_name + "_TEST",
        middle_name=test_user.middle_name + "_TEST" if test_user.middle_name else None,
        code=test_user.code if hasattr(test_user, 'code') else 999999,
        director_first_name=test_user.director_first_name if hasattr(test_user, 'director_first_name') else None,
        director_last_name=test_user.director_last_name if hasattr(test_user, 'director_last_name') else None,
        director_middle_name=test_user.director_middle_name if hasattr(test_user, 'director_middle_name') else None,
        division=test_user.division + "_TEST" if hasattr(test_user, 'division') and test_user.division else "TEST_DIVISION",
        departament=test_user.departament + "_TEST" if hasattr(test_user, 'departament') and test_user.departament else "TEST_DEPARTMENT",
        post=test_user.post + "_TEST" if hasattr(test_user, 'post') and test_user.post else "TEST_POST",
        number="+79990000000",
        works=True,
        departament_code=test_user.departament_OU.code if hasattr(test_user, 'departament_OU') and test_user.departament_OU else 9999,
        division_code=test_user.division_OU.code if hasattr(test_user, 'division_OU') and test_user.division_OU else 9998,
        director_code=test_user.director_obj.code if hasattr(test_user, 'director_obj') and test_user.director_obj else 9997,
        division_OU=test_user.division_OU if hasattr(test_user, 'division_OU') else None,
        departament_OU=test_user.departament_OU if hasattr(test_user, 'departament_OU') else None,
        director_obj=test_user.director_obj if hasattr(test_user, 'director_obj') else None
    )

    logging.info(f"Созданы тестовые данные для пользователя {test_user.login}:")
    logging.info(f"Новое ФИО: {test_data.last_name} {test_data.first_name} {test_data.director_middle_name}")
    logging.info(f"Подразделение: {test_data.division} (код: {test_data.division_code})")
    logging.info(f"Отдел: {test_data.departament} (код: {test_data.division_code})")
    logging.info(f"Должность: {test_data.post}")
    logging.info(f"Телефон: {test_data.number}")

    logging.info("3. Тестирование операций (без реальных изменений):")

    logging.info("Тест обновления ФИО:")
    update_FIO(test_user, test_data)

    logging.info("Тест изменения должности:")
    update_post(test_user, "Новая тестовая должность")

    if ous:
        logging.info("Тест перемещения между подразделениями:")
        test_data.division_OU = ous[-1]
        division_change(test_user, test_data)

    logging.info("=== ТЕСТИРОВАНИЕ ЗАВЕРШЕНО ===")
    logging.info("Для реального выполнения установить dry_run=false в config.json")

if __name__ == "__main__":
    # Для тестирования на реальной системе
    config.dry_run = True  # Гарантируем dry-run режим
    check_real_system()

Overwriting ad.py


In [14]:
%%writefile test.py
from ad import import_users_from_domain, get_all_ou, get_all_groups
from core import config
import logging

def test_real_system():
    logging.info("тестовое подключение к AD")

    # 1. Проверка подключения и чтения
    try:
        groups = get_all_groups()
        ous = get_all_ou(groups)
        users = import_users_from_domain(ous)

        logging.info(f"Найдено {len(users)} пользователей, {len(groups)} групп, {len(ous)} OUs")

        # 2. Выбор тестового пользователя
        test_user = next((u for u in users if u.login == "test_user"), None)
        if not test_user:
            logging.warning("Тестовый пользователь не найден, использовние первого пользователя")
            test_user = users[0]

        logging.info(f"Выбранные тестовые пользователи: {test_user.FIO} ({test_user.login})")

        # 3. Имитация изменений
        from objects import File_user_object

        # Тестовые данные
        test_data = File_user_object(
            first_name="Иван",
            last_name="Иванов",
            middle_name="Иванович",
            code=1001,
            director_first_name="Петр",
            director_last_name="Петров",
            director_middle_name="Петрович",
            division="IT",
            departament="Dev",
            post="Developer",
            number="123",
            works=True,
            departament_code=2001,
            division_code=3001,
            director_code=1002,
            division_OU=None,
            departament_OU=None,
            director_obj=None
        )

        # 4. Dry-run тесты
        from ad import update_FIO, division_change

        logging.info("Тестирование изменения FIO...")
        update_FIO(test_user, test_data)

        logging.info("Тестирование смены подразделения...")
        if ous:
            test_data.division_OU = ous[-1]  # Последнее OU в списке
            division_change(test_user, test_data)

        logging.info("Все dry-run тесты завершены успешно")

    except Exception as e:
        logging.error(f"Тест провален: {str(e)}")

if __name__ == '__main__':
    config.dry_run = True  # Гарантируем dry-run режим
    test_real_system()

Writing test.py
